# K-point and cutoff convergence for Li bcc surfaces

This notebook tests plane-wave cutoff (`ecut`) and k-point mesh convergence for
Li adatom on bcc Li surfaces: (110), (111), and (100).

It assumes you already have relaxed **initial** adatom states for each surface
saved as trajectory files (e.g. from your NEB pre-relaxation). You can adjust
the file paths in the configuration cell below.

In [3]:

import os
import numpy as np
import pandas as pd

from ase.io import read
from gpaw import GPAW, PW

# === User configuration ===
# Adjust these to match your project layout / choices.

# Exchange–correlation functional (set to whatever you used in your NEB)
xc = "PBE"  # e.g. "PBE", "LDA", or a GPAW xc string

# Directory to write GPAW output text files
output_dir = "crystal_structure_neb_kconv"
os.makedirs(output_dir, exist_ok=True)

# Paths to relaxed initial adatom states for each surface.
# TODO: update these paths to your actual files.
state_files = {
    "li110": "crystal_structure_neb_results/li-li-bcc--110--initial.traj",
    "li111": "crystal_structure_neb_results/li-li-bcc--111--initial.traj",
    "li100": "crystal_structure_neb_results/li-li-bcc--100--initial.traj",
}

# Load Atoms objects
states = {name: read(path) for name, path in state_files.items()}
states


{'li110': Atoms(symbols='Li73', pbc=[True, True, False], cell=[14.806815998046307, 10.47, 27.403407999023152], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 8, 9, 10, 11, 16, 17, 18, 19, 24, 25, 26, 27, 32, 33, 34, 35, 40, 41, 42, 43, 48, 49, 50, 51, 56, 57, 58, 59, 64, 65, 66, 67]), calculator=SinglePointCalculator(...)),
 'li111': Atoms(symbols='Li73', pbc=[True, True, False], cell=[[14.806815998046307, 0.0, 0.0], [7.403407999023154, 12.823078803469937, 0.0], [0.0, 0.0, 27.052333538151277]], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 8, 9, 10, 11, 16, 17, 18, 19, 24, 25, 26, 27, 32, 33, 34, 35, 40, 41, 42, 43, 48, 49, 50, 51, 56, 57, 58, 59, 64, 65, 66, 67]), calculator=SinglePointCalculator(...)),
 'li100': Atoms(symbols='Li73', pbc=[True, True, False], cell=[10.47, 10.47, 32.215], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 8, 9, 10, 11, 16, 17, 18, 19, 24, 25, 26, 27, 32, 33, 34, 35, 40, 41, 42, 43, 48, 49, 50, 51, 56, 57, 58, 59, 64, 65, 66, 67]), calculator=S

In [4]:

def sp_energy_with_settings(
    atoms,
    label: str,
    pw_cutoff: float,
    k: int,
    txt_prefix: str = "kconv",
) -> float:
    """Single-point energy with given PW cutoff and kpts=(k, k, 1)."""
    atoms = atoms.copy()  # avoid modifying original
    calc = GPAW(
        mode=PW(pw_cutoff),
        xc=xc,
        kpts=(k, k, 1),
        convergence={"eigenstates": 5e-6, "density": 1e-4},
        txt=os.path.join(
            output_dir,
            f"{txt_prefix}-{label}-ecut{int(pw_cutoff)}-k{k}.txt",
        ),
    )
    atoms.calc = calc
    E = atoms.get_potential_energy()
    return E


In [5]:

# === Convergence grid ===
pw_list = [300, 400, 500, 600]  # eV
k_list  = [2, 3, 4]             # (2x2x1), (3x3x1), (4x4x1)

results = []

for surf_label, atoms in states.items():
    print(f"=== Surface: {surf_label} ===")
    for pw in pw_list:
        for k in k_list:
            E = sp_energy_with_settings(
                atoms,
                label=surf_label,
                pw_cutoff=pw,
                k=k,
                txt_prefix="kconv",
            )
            results.append(
                {
                    "surface": surf_label,
                    "ecut_eV": pw,
                    "kpts": f"{k}x{k}x1",
                    "k": k,
                    "E_eV": E,
                }
            )
            print(f"  ecut={pw:4d} eV, k={k}x{k}x1 -> E = {E:.6f} eV")

df_conv = pd.DataFrame(results)
df_conv


=== Surface: li110 ===
  ecut= 300 eV, k=2x2x1 -> E = -128.642857 eV
  ecut= 300 eV, k=3x3x1 -> E = -129.051134 eV
  ecut= 300 eV, k=4x4x1 -> E = -129.097431 eV
  ecut= 400 eV, k=2x2x1 -> E = -128.650326 eV
  ecut= 400 eV, k=3x3x1 -> E = -129.058228 eV
  ecut= 400 eV, k=4x4x1 -> E = -129.104872 eV
  ecut= 500 eV, k=2x2x1 -> E = -128.657966 eV
  ecut= 500 eV, k=3x3x1 -> E = -129.066075 eV
  ecut= 500 eV, k=4x4x1 -> E = -129.112494 eV
  ecut= 600 eV, k=2x2x1 -> E = -128.659149 eV
  ecut= 600 eV, k=3x3x1 -> E = -129.067331 eV
  ecut= 600 eV, k=4x4x1 -> E = -129.113685 eV
=== Surface: li111 ===
  ecut= 300 eV, k=2x2x1 -> E = -125.212861 eV
  ecut= 300 eV, k=3x3x1 -> E = -125.179991 eV
  ecut= 300 eV, k=4x4x1 -> E = -125.108001 eV
  ecut= 400 eV, k=2x2x1 -> E = -125.219786 eV
  ecut= 400 eV, k=3x3x1 -> E = -125.186895 eV
  ecut= 400 eV, k=4x4x1 -> E = -125.114935 eV
  ecut= 500 eV, k=2x2x1 -> E = -125.226986 eV
  ecut= 500 eV, k=3x3x1 -> E = -125.194037 eV
  ecut= 500 eV, k=4x4x1 -> E = -12

,surface,ecut_eV,kpts,k,E_eV
0,li110,300,2x2x1,2,-128.642857
1,li110,300,3x3x1,3,-129.051134
2,li110,300,4x4x1,4,-129.097431
3,li110,400,2x2x1,2,-128.650326
4,li110,400,3x3x1,3,-129.058228
5,li110,400,4x4x1,4,-129.104872
6,li110,500,2x2x1,2,-128.657966
7,li110,500,3x3x1,3,-129.066075
8,li110,500,4x4x1,4,-129.112494
9,li110,600,2x2x1,2,-128.659149


In [6]:

def analyse_convergence(df: pd.DataFrame, ref_pw: float = 600, ref_k: int = 4) -> pd.DataFrame:
    """Add column with ΔE (meV) vs reference (ref_pw, ref_k) for each surface."""
    rows = []
    for surf in sorted(df["surface"].unique()):
        df_s = df[df["surface"] == surf].copy()
        ref_row = df_s[(df_s["ecut_eV"] == ref_pw) & (df_s["k"] == ref_k)]
        if len(ref_row) != 1:
            print(f"WARNING: reference (ecut={ref_pw}, k={ref_k}) not found for {surf}")
            continue
        E_ref = ref_row["E_eV"].values[0]
        df_s["dE_meV_vs_ref"] = (df_s["E_eV"] - E_ref) * 1000.0
        rows.append(df_s)
    if not rows:
        raise RuntimeError("No reference rows found; adjust ref_pw / ref_k to match your grid.")
    return pd.concat(rows, ignore_index=True)


df_conv_with_delta = analyse_convergence(df_conv, ref_pw=600, ref_k=4)
df_conv_with_delta.sort_values(["surface", "ecut_eV", "k"], inplace=True)
df_conv_with_delta


,surface,ecut_eV,kpts,k,E_eV,dE_meV_vs_ref
0,li100,300,2x2x1,2,-131.405546,422.812264
1,li100,300,3x3x1,3,-131.740208,88.149978
2,li100,300,4x4x1,4,-131.812406,15.951817
3,li100,400,2x2x1,2,-131.412825,415.533004
4,li100,400,3x3x1,3,-131.747458,80.900470
5,li100,400,4x4x1,4,-131.819632,8.725929
6,li100,500,2x2x1,2,-131.420321,408.037472
7,li100,500,3x3x1,3,-131.754989,73.368632
8,li100,500,4x4x1,4,-131.827183,1.174821
9,li100,600,2x2x1,2,-131.421458,406.899975


In [7]:

# Show a compact table of ΔE (meV) vs (ecut, k) for each surface
from IPython.display import display

for surf in ["li110", "li111", "li100"]:
    print(f"\n=== {surf} ===")
    df_surf = df_conv_with_delta[df_conv_with_delta["surface"] == surf]
    if df_surf.empty:
        print("  (no data for this surface; check state_files / convergence grid)")
        continue
    display(
        df_surf
        .pivot(index="ecut_eV", columns="k", values="dE_meV_vs_ref")
        .rename_axis("ecut_eV (PW)", axis=0)
        .rename_axis("k (k x k x 1)", axis=1)
        .round(3)
    )



=== li110 ===


k (k x k x 1),2,3,4
ecut_eV (PW),,,
300,470.829,62.552,16.254
400,463.360,55.458,8.813
500,455.719,47.610,1.191
600,454.536,46.354,0.000



=== li111 ===


k (k x k x 1),2,3,4
ecut_eV (PW),,,
300,-89.563,-56.694,15.297
400,-96.488,-63.598,8.363
500,-103.688,-70.739,1.163
600,-104.848,-71.894,0.000



=== li100 ===


k (k x k x 1),2,3,4
ecut_eV (PW),,,
300,422.812,88.150,15.952
400,415.533,80.900,8.726
500,408.037,73.369,1.175
600,406.900,72.168,0.000
